# Finding the optimal parameters for Historical Linear Regression algorithm (spatial means)

## Importing

In [11]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xskillscore as xs

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import r_regression

from skfda.representation.grid import FDataGrid
from skfda.ml.clustering import KMeans

from skfda.ml.regression import FPLSRegression

from sklearn.metrics import root_mean_squared_error as rmse

import os
import lzma
import dill

import random

import cmocean.cm as cm
import salishsea_tools.viz_tools as sa_vi
from tqdm import tqdm

np.warnings.filterwarnings('ignore') # For the nan mean warning


## Datasets Preparation

In [12]:
# Creation of the training - testing datasets
def datasets_preparation(dataset, dataset2, clusters, name):
    
    indx = np.where((dataset.time_counter.dt.month==2) & (dataset.time_counter.dt.day==29))
    
    targets = dataset[name].to_numpy().reshape(*dataset[name].to_numpy().shape[:1],-1)

    inputs = np.stack([
        dataset2['Summation_of_solar_radiation'].to_numpy().reshape(*dataset2['Summation_of_solar_radiation'].to_numpy().shape[:1],-1),
        dataset2['Mean_air_temperature'].to_numpy().reshape(*dataset2['Mean_air_temperature'].to_numpy().shape[:1],-1),
        dataset2['Mean_pressure'].to_numpy().reshape(*dataset2['Mean_pressure'].to_numpy().shape[:1],-1),
        dataset2['Mean_precipitation'].to_numpy().reshape(*dataset2['Mean_precipitation'].to_numpy().shape[:1],-1),
        dataset2['Mean_specific_humidity'].to_numpy().reshape(*dataset2['Mean_specific_humidity'].to_numpy().shape[:1],-1),
        ])

    # Deleting 29 of February
    inputs = np.delete(inputs,indx,axis=1)
    targets = np.delete(targets,indx,axis=0)

    # Splitting in years
    inputs = np.split(inputs,len(np.unique(dataset.time_counter.dt.year)),axis=1)
    targets = np.split(targets,len(np.unique(dataset.time_counter.dt.year)),axis=0)

    # Means
    inputs = np.nanmean(inputs,axis=0)
    targets = np.nanmean(targets,axis=0)

    x =  np.tile(dataset2.x, len(dataset2.y))
    y =  np.tile(np.repeat(dataset2.y, len(dataset2.x)),1)

    indx = np.where((~np.isnan(targets).any(axis=0)) & (x>10) & ((x>100) | (y<880)))
    inputs = inputs[:,:,indx[0]]
    targets = targets[:,indx[0]]

    clusters = np.tile(np.ravel(clusters), len(dataset.time_counter))
    clusters = clusters[indx[0]]

    return(inputs, targets, indx, clusters)


## Clustering Finalization


In [13]:
def clustering(dataset,quant,indx,name):

    # Training
    n_clusters = 6
    kmeans = KMeans(n_clusters=n_clusters)
    clusters = kmeans.fit_predict(quant)

    # Sorting so that cluster 1 has the minimum mean target value, 6 the maximum

        # Finding the mean of each cluster
    if name == 'inputs':
        cluster_mean_all = np.mean(kmeans.cluster_centers_.data_matrix,axis=1)
        cluster_mean = cluster_mean_all[:,0]  # Sorted based on the first input
    else:
        cluster_mean = np.squeeze(np.mean(kmeans.cluster_centers_.data_matrix,axis=1))

        # The index to sort the clusters
    indx3 = np.argsort(np.argsort(cluster_mean)) # For the complete map we need the double np.argsort

        # Sorting
    for j in np.arange(0,len(np.unique(clusters))):
        clusters = xr.where(kmeans.labels_==j, indx3[j], clusters)

    unique, _ = np.unique(clusters, return_counts=True)

    # Creating the map
    indx2 = np.full(len(dataset.y) * len(dataset.x),np.nan)
    indx2[indx[0]] = clusters
    clusters = np.reshape(indx2,(len(dataset.y),len(dataset.x))) 
    clusters2 = xr.DataArray(clusters,dims = ['y','x'])
    
    # Plotting
    fig, ax = plt.subplots(figsize =(5,9))

    cmap = plt.get_cmap('tab20', unique.max()+1)
    cmap.set_bad('gray')
    clus = clusters2.plot(ax=ax, cmap=cmap, vmin = unique.min(), vmax = unique.max()+1, add_colorbar=False)

    cbar = fig.colorbar(clus, ticks = unique+0.5) 
    cbar.set_ticklabels(unique+1)
    cbar.set_label('Clusters [count]')
    ax.set_title('Functional Clustering for '+ name + ' (2007-2020)')

    sa_vi.set_aspect(ax)
    plt.show()

    return(clusters,n_clusters)


## Functional Clustering (Target)

In [14]:
def func_clust_target(dataset, name):

    indx = np.where((dataset.time_counter.dt.month==2) & (dataset.time_counter.dt.day==29))
    
    targets = dataset[name].to_numpy().reshape(*dataset[name].to_numpy().shape[:1],-1)

    # Deleting 29 of February
    targets = np.delete(targets,indx,axis=0)

    # Splitting in years
    targets = np.split(targets,len(np.unique(dataset.time_counter.dt.year)),axis=0)

    # Means
    targets = np.nanmean(targets,axis=0)

    x =  np.tile(dataset.x, len(dataset.y))
    y =  np.tile(np.repeat(dataset.y, len(dataset.x)),1)

    indx = np.where((~np.isnan(targets).any(axis=0))& (x>10) & ((x>100) | (y<880)))
    targets = targets[:,indx[0]]

    # Converting it to an appropriate format for functional clustering
    targets = targets.transpose()
    targets2 = FDataGrid(targets)

    clusters, n_clusters = clustering(dataset,targets2,indx,name)

    return(clusters,0,n_clusters)


## Functional Clustering (Drivers)

In [15]:
def func_clust_drivers(dataset,dataset2,name):

    indx = np.where((dataset2.time_counter.dt.month==2) & (dataset2.time_counter.dt.day==29))

    inputs = np.stack([
        dataset2['Summation_of_solar_radiation'].to_numpy().reshape(*dataset2['Summation_of_solar_radiation'].to_numpy().shape[:1],-1),
        dataset2['Mean_air_temperature'].to_numpy().reshape(*dataset2['Mean_air_temperature'].to_numpy().shape[:1],-1),
        dataset2['Mean_pressure'].to_numpy().reshape(*dataset2['Mean_pressure'].to_numpy().shape[:1],-1),
        dataset2['Mean_precipitation'].to_numpy().reshape(*dataset2['Mean_precipitation'].to_numpy().shape[:1],-1),
        dataset2['Mean_specific_humidity'].to_numpy().reshape(*dataset2['Mean_specific_humidity'].to_numpy().shape[:1],-1),
        ])
    
    targets = dataset['Diatom'].to_numpy().reshape(*dataset['Diatom'].to_numpy().shape[:1],-1)

    # Deleting 29 of February
    inputs = np.delete(inputs,indx,axis=1)
    targets = np.delete(targets,indx,axis=0)

    # Splitting in years
    inputs = np.split(inputs,len(np.unique(dataset2.time_counter.dt.year)),axis=1)
    targets = np.split(targets,len(np.unique(dataset.time_counter.dt.year)),axis=0)

    # Means
    inputs = np.nanmean(inputs,axis=0)
    targets = np.nanmean(targets,axis=0)

    x =  np.tile(dataset2.x, len(dataset2.y))
    y =  np.tile(np.repeat(dataset2.y, len(dataset2.x)),1)

    indx = np.where((~np.isnan(targets).any(axis=0))& (x>10) & ((x>100) | (y<880))) # Target goes down to 100m
    inputs = inputs[:,:,indx[0]]

    # Scaling the inputs
    temp = np.reshape(inputs,(len(inputs),inputs.shape[1]*inputs.shape[2]))
    temp = temp.transpose()
    scaler_inputs = make_column_transformer((StandardScaler(), [0,1,2,3,4]))
    temp = scaler_inputs.fit_transform(temp)
    temp = temp.transpose()
    inputs = np.reshape(temp,(len(inputs),inputs.shape[1],inputs.shape[2])) 

    # Converting it to an appropriate format for functional clustering
    inputs = np.transpose(inputs,axes=(2,1,0))
    inputs2 = FDataGrid(inputs, np.arange(0,len(inputs[0])))

    clusters, n_clusters = clustering(dataset2,inputs2,indx,name)

    return(clusters, 1, n_clusters)


## Regressor

In [16]:
def regressor (inputs, targets,name,i,lag):

    temp_inputs = np.reshape(inputs,(len(inputs),inputs.shape[1]*inputs.shape[2]))
    temp_inputs = temp_inputs.transpose()
    temp_targets = np.ravel(targets)

    # Scaling the inputs
    scaler_inputs = make_column_transformer((StandardScaler(), [0,1,2,3,4]))
    temp_inputs = scaler_inputs.fit_transform(temp_inputs)
    temp_inputs = temp_inputs.transpose()
    inputs = np.reshape(temp_inputs,(len(inputs),inputs.shape[1],inputs.shape[2]))   
    inputs = np.transpose(inputs,axes=(2,1,0))
    
    # Scaling the targets
    scaler_targets = StandardScaler()
    temp_targets = np.expand_dims(temp_targets,-1)
    temp_targets = scaler_targets.fit_transform(temp_targets)
    targets = temp_targets.reshape(targets.shape)

     # Final transformations
    targets = targets.transpose()
    inputs = FDataGrid(data_matrix=inputs, grid_points=np.arange(0,len(targets[0])))
    targets = FDataGrid(data_matrix=targets, grid_points=np.arange(0,len(targets[0])))

    ## Smoothing
    from skfda.representation.basis import VectorValuedBasis,FourierBasis

    # x_basis = np.tile(FourierBasis(domain_range=(0,74),n_basis=10),5)
    # basis = VectorValuedBasis(x_basis)
    # inputs = inputs.to_basis(basis)
    
    # targets = targets.to_basis(FourierBasis(n_basis=10))

    from skfda.ml.regression import HistoricalLinearRegression,LinearRegression,FPCARegression

    model = HistoricalLinearRegression(n_intervals=3,lag=lag)
    regr = model.fit(inputs,targets)

    return(regr,scaler_inputs,scaler_targets)


## Pre-training

In [17]:
def pre_training(dataset,dataset2,n_clusters,clusters0,name):

    np.warnings.filterwarnings('ignore') # For the nan mean warning

    clusters_indiv_t = np.zeros((len(np.unique(dataset.time_counter.dt.dayofyear))-1,len(np.unique(dataset.time_counter.dt.year)),n_clusters))
    clusters_indiv_d = np.zeros((5,len(np.unique(dataset.time_counter.dt.dayofyear))-1,len(np.unique(dataset.time_counter.dt.year)),n_clusters))

    ds = dataset
    ds2 = dataset2

    for i in range(0, len(np.unique(ds.time_counter.dt.year))):

        dataset = ds.sel(time_counter = slice(str(np.unique(ds.time_counter.dt.year)[i]), str(np.unique(ds.time_counter.dt.year)[i])))
        dataset2 = ds2.sel(time_counter = slice(str(np.unique(ds2.time_counter.dt.year)[i]), str(np.unique(ds2.time_counter.dt.year)[i])))

        inputs, targets, indx, _ = datasets_preparation(dataset, dataset2, clusters0, name)

        inputs1 = np.squeeze(inputs)
        targets1 = np.squeeze(targets)
        clusters1 = np.ravel(clusters0)[indx]

        for j in range (0,n_clusters):

            temp = xr.where(clusters1==j, inputs1, np.nan)
            clusters_indiv_d[:,:,i,j] = np.nanmean(temp,axis=2)

            temp = xr.where(clusters1==j, targets1, np.nan)
            clusters_indiv_t[:,i,j] = np.nanmean(temp,axis=1)

    return(clusters_indiv_d,clusters_indiv_t,indx)


## Scaling

In [18]:
def scaling(regr,inputs,scaler_inputs,targets,scaler_targets):

    # Scaling the inputs
    temp = np.reshape(inputs,(len(inputs),inputs.shape[1]*inputs.shape[2]))
    temp = temp.transpose()
    temp = scaler_inputs.transform(temp)
    temp = temp.transpose()        
    inputs = np.reshape(temp,(len(inputs),inputs.shape[1],inputs.shape[2]))
        
    inputs = np.transpose(inputs,axes=(2,1,0))
    inputs = FDataGrid(data_matrix=inputs, grid_points=np.arange(0,len(targets)))

    predictions = regr.predict(inputs)

    # Post-processing of predictions
    predictions = np.array(predictions.to_grid(np.arange(0,len(targets))).data_matrix)
    predictions = np.squeeze(predictions,2)

    # Scaling the predictions
    temp = np.ravel(predictions)
    temp = np.expand_dims(temp,axis=-1)
    temp = scaler_targets.inverse_transform(temp)
    predictions = temp.reshape(predictions.shape)
    predictions = predictions.transpose()

    return(predictions)


## Plotting (Mean Values)

In [19]:
def plotting_mean_values(dates,mean_targets,mean_predictions,category,units,region):

    years = np.unique(dates.year)
    ticks = [0]
    
    fig, _ = plt.subplots(figsize=(19,5))
    
    mean_targets = np.ma.array(mean_targets)
    mean_predictions = np.ma.array(mean_predictions)

    for year in years[:-1]:
        ticks.append((np.where(dates.year==year)[0][-1]+1))
        mean_targets[(np.where(dates.year==year)[0][-1]+1)] = np.ma.masked
        mean_predictions[(np.where(dates.year==year)[0][-1]+1)] = np.ma.masked

    plt.plot(mean_targets, label = 'targets')
    plt.plot(mean_predictions, label = 'predictions')
    plt.xlabel('Years')
    plt.xticks(ticks,years)
    plt.suptitle('Mean '+category + ' ' +units + ' (15 Feb - 30 Apr) ' + region)
    plt.legend()
    
    fig.show()


## Initiation

In [ ]:
name = 'Flagellate'
units = '[mmol m-2]'
category = 'Concentrations'

ds = xr.open_dataset('/data/ibougoudis/MOAD/files/integrated_original.nc')
ds2 = xr.open_dataset('/data/ibougoudis/MOAD/files/external_inputs.nc')

# ds = ds.isel(y=(np.arange(ds.y[0], ds.y[-1], 5)), 
#     x=(np.arange(ds.x[0], ds.x[-1], 5)))

# ds2 = ds2.isel(y=(np.arange(ds2.y[0], ds2.y[-1], 5)), 
#     x=(np.arange(ds2.x[0], ds2.x[-1], 5)))

dataset = ds.sel(time_counter = slice('2007', '2020'))
dataset2 = ds2.sel(time_counter = slice('2007', '2020'))

dataset_test = ds.sel(time_counter = slice('2021', '2024'))
dataset2_test = ds2.sel(time_counter = slice('2021', '2024'))

years = np.unique(dataset.time_counter.dt.year)

# Selecting the clustering input (drivers or target)
clusters0, id, n_clusters = func_clust_target(dataset, name)
# clusters0, id, n_clusters = func_clust_drivers(dataset,dataset2,'inputs')

inputs,targets,indx = pre_training(dataset,dataset2,n_clusters,clusters0,name)

inputs_test,targets_test,indx = pre_training(dataset_test, dataset2_test, n_clusters, clusters0, name)


## Training

In [ ]:
r_train = np.zeros((n_clusters,targets.shape[0]))
rms_train = np.zeros((n_clusters,targets.shape[0]))
slope_train = np.zeros((n_clusters,targets.shape[0]))

r_test = np.zeros((n_clusters,targets.shape[0]))
rms_test = np.zeros((n_clusters,targets.shape[0]))
slope_test = np.zeros((n_clusters,targets.shape[0]))

for i in range (0,n_clusters):

    inputs2 = inputs[:,:,:,i] # inputs of the i cluster
    targets2 = targets[:,:,i] # targets of the i cluster

    inputs2_test = inputs_test[:,:,:,i] # inputs of the i cluster
    targets2_test = targets_test[:,:,i] # targets of the i cluster

    for lag in tqdm(range(1,76)):

        regr, scaler_inputs,scaler_targets = regressor(inputs2,targets2,name,i,lag)

        predictions = scaling(regr,inputs2,scaler_inputs,targets2,scaler_targets) # putting them in the right place

        r_train[i,lag-1] = np.corrcoef(np.ravel(targets2),np.ravel(predictions))[0][1]
        rms_train[i,lag-1] = rmse(np.ravel(targets2),np.ravel(predictions))
        m,_ = np.polyfit(np.ravel(targets2), np.ravel(predictions), deg=1)
        slope_train[i,lag-1]= np.round(m,3)

        predictions = scaling(regr,inputs2_test,scaler_inputs,targets2_test,scaler_targets)

        r_test[i,lag-1] = np.corrcoef(np.ravel(targets2_test),np.ravel(predictions))[0][1]
        rms_test[i,lag-1] = rmse(np.ravel(targets2_test),np.ravel(predictions))
        m,_ = np.polyfit(np.ravel(targets2_test), np.ravel(predictions), deg=1)
        slope_test[i,lag-1] = np.round(m,3)

    print('The best correlation coefficient for training for cluster ' +str(i+1)+ ' is with number of interval: ' +str(np.argmax(r_train[i])+1))
    print('The best root mean square error for training for cluster ' +str(i+1)+ ' is with number of interval: ' +str(np.argmin(rms_train[i])+1))

    print('The best correlation coefficient for testing for cluster ' +str(i+1)+ ' is with number of interval: ' +str(np.argmax(r_test[i])+1))
    print('The best root mean square error for testing for cluster ' +str(i+1)+ ' is with number of interval: ' +str(np.argmin(rms_test[i])+1))


In [ ]:
r_train